In [4]:
import pandas as pd
import numpy as np

6.1文本格式数据的读写
由于现实世界的数据非常混乱，随着事件推移，一些数据夹杂自函数（尤其是read_csv）的可选参数变得非常复杂。pandas在线文档有大量的示例展示这些参数是如何工作的。

In [9]:
%pwd
df1 = pd.read_csv('examples/ex1.csv')
print(df1)

# 使用read_table，并指定分割符
df2 = pd.read_table('examples/ex1.csv', sep=',')
print(df2)

# 有的文件不不包含表头行，可以允许pandas自动分配默认列名，也可以自己指定列名
df3 = pd.read_csv('examples/ex2.csv', header=None)
print(df3)
df4 = pd.read_csv('examples/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])
print(df4)

# 指定某一位置的列为索引
names = ['a', 'b', 'c', 'd', 'message']
df5 = pd.read_csv('examples/ex2.csv', names=names, index_col='message')
print(df5)

# 当你需要从多个列中形成一个分层所有，需要传入一个包含序列号或列名的列表
parsed = pd.read_csv('examples/csv_mindex.csv', index_col=['key1','key2'])
print(parsed)

   a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo
   a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo
   0   1   2   3      4
0  1   2   3   4  hello
1  5   6   7   8  world
2  9  10  11  12    foo
   a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo
         a   b   c   d
message               
hello    1   2   3   4
world    5   6   7   8
foo      9  10  11  12
           value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16


In [18]:
# 当字段是以不同数量的空格分开时，可以通过向read_table传入一个正则表达式作为分隔符
result = pd.read_table('examples/ex3.txt', sep='\s+')
print(result)

# 可以使用skiprows 来跳过某些行
print(pd.read_csv('examples/ex4.csv', skiprows=[0, 2, 3]))

# 缺失值处理
# 默认情况下，pandas使用常见的标识，例如NA和NULL
print(pd.read_csv('examples/ex5.csv'))
# na_values选项可以传入一个列表或者一组字符串来处理缺失值，即需要用NA替换的值序列
print(pd.read_csv('examples/ex5.csv', na_values=['NULL']))
# 在字典中，每列可以指定不同的缺失值标识
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
print(pd.read_csv('examples/ex5.csv',na_values=sentinels))

            A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491
   a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo
  something  a   b     c   d message
0       one  1   2   3.0   4     NaN
1       two  5   6   NaN   8   world
2     three  9  10  11.0  12     foo
  something  a   b     c   d message
0       one  1   2   3.0   4     NaN
1       two  5   6   NaN   8   world
2     three  9  10  11.0  12     foo
  something  a   b     c   d message
0       one  1   2   3.0   4     NaN
1       NaN  5   6   NaN   8   world
2     three  9  10  11.0  12     NaN


6.1.1 分块读入文本文件
当处理大型文件或找出正确的参数集来正确处理大文件时，可能需要读入文件的一个小片段或者按小块遍历文件

In [30]:
# 在尝试大文件之前，我们可以先对pandas的显示设置进行调整，使之更为紧凑
pd.options.display.max_rows = 10
result = pd.read_csv('examples/ex6.csv')
print(result)

# 读取一小部分（避免读取整个文件），可以知名nrows
print(pd.read_csv('examples/ex6.csv', nrows=5))

# 为了分块读入文件，可以指定chunksize作为每一块的行数
chunksize = pd.read_csv('examples/ex6.csv', chunksize=1000)
print(type(chunksize))
# read_csv返回的TextParser对象允许你根据chunksize遍历文件。
# 例如，便利ex6.csv，并对‘key’列聚合获得计数值
tot = pd.Series([])
for piece in chunksize:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)
    
tot = tot.sort_values(ascending=False)
print(tot[:10])

           one       two     three      four key
0     0.467976 -0.038649 -0.295344 -1.824726   L
1    -0.358893  1.404453  0.704965 -0.200638   B
2    -0.501840  0.659254 -0.421691 -0.057688   G
3     0.204886  1.074134  1.388361 -0.982404   R
4     0.354628 -0.133116  0.283763 -0.837063   Q
...        ...       ...       ...       ...  ..
9995  2.311896 -0.417070 -1.409599 -0.515821   L
9996 -0.479893 -0.650419  0.745152 -0.646038   E
9997  0.523331  0.787112  0.486066  1.093156   K
9998 -0.362559  0.598894 -1.843201  0.887292   G
9999 -0.096376 -1.012999 -0.657431 -0.573315   0

[10000 rows x 5 columns]
        one       two     three      four key
0  0.467976 -0.038649 -0.295344 -1.824726   L
1 -0.358893  1.404453  0.704965 -0.200638   B
2 -0.501840  0.659254 -0.421691 -0.057688   G
3  0.204886  1.074134  1.388361 -0.982404   R
4  0.354628 -0.133116  0.283763 -0.837063   Q
<class 'pandas.io.parsers.TextFileReader'>
E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J

6.1.2 将数据写入文本格式

In [44]:
# 读取CSV文件
data = pd.read_csv('examples/ex5.csv')
print(data)

# 使用DataFrame的to_csv方法，我们可以将数据到处为逗号分隔的文件
data.to_csv('examples/out.csv')

# 使用其他的分隔符（写入到sys.stdout）
import sys
data.to_csv(sys.stdout, sep='|')

# 缺失值在输出时以空白字符串出现，可以用其他标识值对缺失值进行标注
data.to_csv(sys.stdout, na_rep='NaN')

# 如果没有其它选项被指定的话，行和列的标签都会被写入，不过二者也都可以禁止
data.to_csv(sys.stdout, index=False, header=False)

# 仅仅写入列的自己，并且按照选择的顺序写入
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

# Series也有to_csv方法
dates = pd.date_range('27/9/2018', periods=7)
ts = pd.Series(np.arange(7), index=dates)
ts.to_csv(sys.stdout)

  something  a   b     c   d message
0       one  1   2   3.0   4     NaN
1       two  5   6   NaN   8   world
2     three  9  10  11.0  12     foo
|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo
,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo
one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo
a,b,c
1,2,3.0
5,6,
9,10,11.0
2018-09-27,0
2018-09-28,1
2018-09-29,2
2018-09-30,3
2018-10-01,4
2018-10-02,5
2018-10-03,6


6.1.3 使用分割格式
绝大多数的表型数据都可以使用函数pandas.readd_table从硬盘中读取。然而，在某些情况下，一些手动操作时必不可少的。接收一个带有一行或多行错误的文件并不少见，read_table也无法解决这种情况。

In [48]:
import csv

# 将文件读取为行的列表
with open('examples/ex7.csv') as f:
    lines = list(csv.reader(f))

# 将数据拆分为列名行和数据行
header, values = lines[0], lines[1:]

# 使用字典推到是和表达式zip(*values)生成一个包含数据列的字典，字典中行专制成列
data_dict = {h: v for h, v in zip(header, zip(*values))}
print(data_dict)

# CSV文件有多种不同分格。如需根据不同的分隔符、字符串引用约定或行中止符定义一种新的格式时，我们可以使用CSV_Dialect定义一个简单的子类。
class my_dialect(csv.Dialect):
    lineterminator = '\n' # 行终止符，默认'\r\n'，读取器会忽略行中止符兵识别跨平台行中止符
    delimiter = ';' # 一个用于分割字段的字符，默认是','
    quotechar = '"' # 用在含有特殊字符字段中的引号，默认是'"'
    quoting = csv.QUOTE_MINIMAL # 引用管理。选项包括csv.QUOTE_ALL(引用所有字段)，csv.QUOTE_MINIMAL(只使用特殊符号，如分隔符)，csv.QUOTE_NONNUMERIC和csv.QUOTE_NONE(不引用)。细节参考Python文档，默认是QUOTE_MINIMAL
    skipinitialspace = False # 忽略每个分隔符后的空白，默认是False
    doublequote = False # 如何处理字段内部引号。如果是True，则是双引号（完整细节和行为请参考在线文档）
    # escapechar = None # 当引用设置为csv.QUOTE_NONE时用于转义分隔符的字符串，默认禁用 

with open('examples/mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}
